### Import all the libraries


In [0]:
# Import all the libraries

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

### Mounting To Get Data From Azure Blob Storage

In [0]:
# Mounting Using Azure Blob Storage to get the data

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX", # Add the client-id here
"fs.azure.account.oauth2.client.secret": 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX', # Add the client-secret here
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/oauth2/token"} # Add client endpoint here


dbutils.fs.mount(
source = "abfss://olympic2021tokyodata@tokyoolympicsice2021.dfs.core.windows.net/", # Container @ Storageaccount
mount_point = "/mnt/tokyoolymic3",
extra_configs = configs)
  

Out[10]: True

### Check the mount point and files

In [0]:
# Check the mounted files with the infer schema option

athletes = spark.read.csv("/mnt/tokyoolymic3/raw-data/athletes.csv", header = True, inferSchema = True)
coaches = spark.read.csv("/mnt/tokyoolymic3/raw-data/coaches.csv", header = True, inferSchema = True)
gender = spark.read.csv("/mnt/tokyoolymic3/raw-data/entriesgenders.csv", header = True,)
medals = spark.read.csv("/mnt/tokyoolymic3/raw-data/medals.csv", header = True, inferSchema = True)
teams = spark.read.csv("/mnt/tokyoolymic3/raw-data/teams.csv", header = True, inferSchema = True)

In [0]:
# Show the details of the athletes file
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
# Print the Schema of the athletes file
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
# Show the details of the coaches file
coaches.show()
  

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [0]:
# Show the details of the gender file
gender.show()


+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
# Print the Schema of the gender file
gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
# Change the data type of the columns

gender = gender.withColumn("Female", col("Female").cast(IntegerType()))
gender = gender.withColumn("Male",col("Male").cast(IntegerType()))
gender = gender.withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
# Print the Schema of the transformed gender file
gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
# Show the details of the medals file
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
#Print the schema of the medals file
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
#Show the details of the teams file
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
#Print the schema of the teams file
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Check top countries by Gold medals
topmedals = medals.orderBy("Gold", ascending=False).select("TeamCountry", "Gold").show()    

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|              Canada|   7|
|              Brazil|   7|
|         New Zealand|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# To write the transformed athletes file to a CSV file in the transformed data files
athletes.write.option("header", "true").csv("/mnt/tokyoolymic3/transformed-data/athletes3")

In [0]:
# To overwrite an existing file, use mode=overwrite

#athletes.write.option("header", "true").csv("/mnt/tokyoolymic/transformed-data/athletes", mode="overwrite")

In [0]:
#To write the transformed coaches file to a CSV file in the transformed data files

coaches.write.option("header", "true").csv("/mnt/tokyoolymic3/transformed-data/coaches3", )

In [0]:
# To write the transformed medals file to a CSV file in the transformed data files
medals.write.option("header", "true").csv("/mnt/tokyoolymic/transformed-data/medals3", )

In [0]:
#Check the help functions
dbutils.help()

This module provides various utilities for users to interact with the rest of Databricks.
 credentials: DatabricksCredentialUtils -> Utilities for interacting with credentials within notebooks data: DataUtils -> Utilities for understanding and interacting with datasets (EXPERIMENTAL) fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console jobs: JobsUtils -> Utilities for leveraging jobs features library: LibraryUtils -> Utilities for session isolated libraries meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

In [0]:

# Carry some data manipulations

averageOfTotal = medals.withColumn("Average", col("Total")/3).select("TeamCountry", "Average")
averageOfTotal.show()

medals.show()

+--------------------+------------------+
|         TeamCountry|           Average|
+--------------------+------------------+
|United States of ...|37.666666666666664|
|People's Republic...|29.333333333333332|
|               Japan|19.333333333333332|
|       Great Britain|21.666666666666668|
|                 ROC|23.666666666666668|
|           Australia|15.333333333333334|
|         Netherlands|              12.0|
|              France|              11.0|
|             Germany|12.333333333333334|
|               Italy|13.333333333333334|
|              Canada|               8.0|
|              Brazil|               7.0|
|         New Zealand| 6.666666666666667|
|                Cuba|               5.0|
|             Hungary| 6.666666666666667|
|   Republic of Korea| 6.666666666666667|
|              Poland| 4.666666666666667|
|      Czech Republic|3.6666666666666665|
|               Kenya|3.3333333333333335|
|              Norway|2.6666666666666665|
+--------------------+------------

In [0]:
# medals.repartition(1).write.option("header", "true").csv("/mnt/tokyoolymic3/transformed-data/medals3", )
# To repartition the data, use the repartition() method.